In [1]:
!pip install bitsandbytes accelerate transformers -qq -U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2024.10.1 requires transformers<4.45.0, but you have transformers 4.45.2 which is incompatible.
unsloth-zoo 2024.10.1 requires transformers<4.45.0, but you have transformers 4.45.2 which is incompatible.


In [2]:
!git clone https://github.com/werserk/HSE-AI-Assistant-Hack-Python-baseline/

fatal: destination path 'HSE-AI-Assistant-Hack-Python-baseline' already exists and is not an empty directory.


In [3]:
import sys
sys.path.append('/content/HSE-AI-Assistant-Hack-Python-baseline')

In [6]:
import torch

torch.cuda.empty_cache()

In [8]:
!nvidia-smi

Wed Oct 16 16:24:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:B7:00.0 Off |                    0 |
| N/A   40C    P0             125W / 700W |  74149MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = 'Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    "Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24",
    device_map='cuda',
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 140.00 MiB. GPU 0 has a total capacity of 79.33 GiB of which 77.25 MiB is free. Process 799314 has 72.40 GiB memory in use. Process 962366 has 6.84 GiB memory in use. Of the allocated memory 6.33 GiB is allocated by PyTorch, and 3.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"}
]

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe(messages)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant', 'content': 'I am an AI language'}]}]

In [8]:
pipe(messages, max_new_tokens=100)

[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I am an AI language model developed by OpenAI, known as ChatGPT. My purpose is to understand and generate human-like text based on the input I receive. I'm designed to assist with a wide range of tasks, from answering questions to providing explanations, and even helping with creative writing. I don't have personal experiences or consciousness, but I'm here to help you with information and tasks to the best of my ability. If you have any specific questions or need assistance with something, feel"}]}]

## Data analysis (train)

In [9]:
import pandas as pd
import numpy as np

In [12]:
!pip install openpyxl -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
path = "/home/jovyan/"

In [11]:
train_solutions = pd.read_excel(path + 'train/solutions.xlsx')
train_solutions = train_solutions.reset_index(drop=True)

In [12]:
train_tasks = pd.read_excel(path + 'train/tasks.xlsx')
train_tasks = train_tasks.reset_index(drop=True)

In [13]:
train_tests = pd.read_excel(path + 'train/tests.xlsx')
train_tests = train_tests.reset_index(drop=True)

In [14]:
print(train_solutions.shape, train_tasks.shape, train_tests.shape)

(347, 5) (22, 4) (105, 6)


In [15]:
solution1 = train_solutions.loc[(train_solutions['task_id'] == 39) & (train_solutions['id'] == 619), 'student_solution'].item()
solution1

'agencies = input().split()\nus = []\n\nfor agency in agencies:\n    if agency.endswith(ум):\n        us.append(agency)\n        \nprint(sorted(us))'

In [16]:
task1 = train_tasks[train_tasks['id']==39]['description'].iloc[0]
task1

'Реализуйте следующую программу:\n\nВ переменную agencies считывается список названий агентств. Этот код уже написан.\nПрограмма проверяет, какие названия агентств оканчиваются на "ум". Если такие строки есть, то программа сохраняет их в отдельный список.\nВ конце программа печатает отсортированный в алфавитном порядке список с названиями агентств, которые были туда сохранены.\nЕсли таких строк нет, программа печатает пустой список.'

In [17]:
comment = train_solutions[(train_solutions['task_id']==39) & (train_solutions['id'] == 619)]['author_comment'].item()
comment

'Вы забыли заключить "ум" в каквычки.'

### WORKING ZONE, DANILA ZDES

In [18]:
system_prompt = """Ты преподаватель Высшей школы экономики. Твоя задача, получив ЗАДАЧУ и РЕШЕНИЕ дать подсказку ученику, где находится ошибка в его РЕШЕНИИ. Прямо напиши где ошибка.

Не пиши большой ответ. Пиши только подсказку, которая поможет студенту двигаться в правильном направлении, но не видеть весь ответ сразу. Люди не должны догадаться, что ты робот.

Пример:
Ввод:
ЗАДАЧА:
Реализуйте следующую программу:\n\nВ переменную agencies считывается список названий агентств. Этот код уже написан.\nПрограмма проверяет, какие названия агентств оканчиваются на "ум". Если такие строки есть, то программа сохраняет их в отдельный список.\nВ конце программа печатает отсортированный в алфавитном порядке список с названиями агентств, которые были туда сохранены.\nЕсли таких строк нет, программа печатает пустой список.
РЕШЕНИЕ СТУДЕНТА (НЕ СМОТРИ НА СПЕЦИАЛЬНЫЕ СИМВОЛЫ):
agencies = input().split()\nus = []\n\nfor agency in agencies:\n    if agency.endswith(ум):\n        us.append(agency)\n        \nprint(sorted(us))
ОЖИДАЕМЫЙ ОТВЕТ:
Вы забыли заключить "ум" в кавычки.
"""
prompt = f"""ЗАДАЧА: {task1}, РЕШЕНИЕ СТУДЕНТА: {solution1}"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]
answer = pipe(messages, max_length=4096)[0]['generated_text'][-1]['content']
print(answer)

Вы забыли заключить "ум" в кавычки.


In [19]:
# соберем наш датасет
final_train_df = pd.read_csv(path + 'train_database.csv')
train_solutions_df = pd.read_excel(path + 'train/solutions.xlsx')

In [20]:
# по train_solutions_df идем и подтягиваем по task_id данные из final_train_df
description = 'a'
input = 'a'
output = 'a'
student_solution = 'a'
author_solution = 'a'
author_comment = 'a'
test_type = 'a'

res = []
for i in range(len(input)): # input из колонки input в new_df
  res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {test_type} \n<Ввод>: {input} \n<Вывод>: {output}")

result = ' '.join(res)
prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n<Комментарий эксперта>: {author_comment}"
print(prompt)

<Task description>: a 
<Example 1>: 
<Тип теста 1>: a 
<Ввод>: a 
<Вывод>: a 
<Решение студента>: a 
<Идеальное решение>: a 
<Комментарий эксперта>: a


In [21]:
final_train_df.shape, train_solutions_df.shape

((105, 9), (347, 5))

In [22]:
final_train_df.nunique()

id                 105
number               6
task_id             22
type                 2
input               88
output              74
level                2
description         22
author_solution     22
dtype: int64

In [23]:
final_train_df.sample(2)

,id,number,task_id,type,input,output,level,description,author_solution
32,105,0,18,open,"['Круглов', 'Квадратов', 'Иванов', 'Марков', '...",Круглов,B,"Напишите функцию, определяющую дизайнера, кото...","def designer (designers, sizes, towns):\n s..."
45,136,1,23,closed,NaN,#FA8072; #DC143C; #CD5C5C,C,"Роман изучает отчет по объекту, где упоминаютс...","punctuation = '!$%&()*+,-./:;<=>?@^_'\n\ndata ..."


In [24]:
train_solutions_df.sample(2)

,id,task_id,student_solution,author_comment,author_comment_embedding
288,613,38,winners = []\n\n# ваш код\n\nwhile True:\n ...,Некорректная табуляция функции print().,-0.37104499340057373 0.10956292599439621 -0.18...
243,544,34,def text_check(data):\n counter = 0\n su...,"Синтаксическая ошибка. Значение ""переведена"" с...",-0.5314068794250488 0.0999922975897789 -0.3069...


In [25]:
train_solutions_df.nunique()

id                          347
task_id                      22
student_solution            347
author_comment              225
author_comment_embedding    220
dtype: int64

In [26]:
new_df = {'id': [], 'task_id': [], 'description': [],
          'input': [], 'output': [], 'student_solution': [],
          'author_solution': [], 'author_comment': [], 'type': []}

In [27]:
for index, row in train_solutions_df.iterrows():
    new_df['id'].append(row['id'])
    new_df['task_id'].append(row['task_id'])
    new_df['student_solution'].append(row['student_solution'])
    new_df['author_comment'].append(row['author_comment'])
    inp = final_train_df[final_train_df['task_id'] == row['task_id']]['input'].tolist()
    out = final_train_df[final_train_df['task_id'] == row['task_id']]['output'].tolist()
    new_df['input'].append(inp)
    new_df['output'].append(out)
    test_type = final_train_df[final_train_df['task_id'] == row['task_id']]['type'].tolist()
    new_df['type'].append(test_type)
    description = final_train_df[final_train_df['task_id'] == row['task_id']]['description'].tolist()[0]
    new_df['description'].append(description)
    author_solution = final_train_df[final_train_df['task_id'] == row['task_id']]['author_solution'].tolist()[0]
    new_df['author_solution'].append(author_solution)

In [28]:
new_df = pd.DataFrame(new_df)

In [29]:
new_df.shape

(347, 9)

In [30]:
new_df.sample(3)

,id,task_id,description,input,output,student_solution,author_solution,author_comment,type
31,44,3,"Реализуйте программу, которая напечатает хэште...",[Выучите китайский язык без усилий выполняя за...,"[#the_best_app, #question, #answer, nan, #счас...",logo = input()\n\nres = []\nfor i in logo.spli...,logo = input()\n\nres = []\nfor info in logo.s...,Ошибка в определении переменной. Переменная in...,"[open, open, open, closed, closed, closed]"
102,288,18,"Напишите функцию, определяющую дизайнера, кото...","[['Круглов', 'Квадратов', 'Иванов', 'Марков', ...","[Круглов, Елисеев, Иванов, Елисеев, Петров, Ст...","def designer (designers, sizes, towns):\n s...","def designer (designers, sizes, towns):\n s...",Ошибка при передаче аргумента функции range. С...,"[open, open, closed, closed, closed, closed]"
180,445,28,"Студент хочет определить языки, которые он буд...","[Английский, Французский, Немецкий, Китайский,...","[Французский; Китайский; Английский, nan, тайс...","course = input().split(', ')\nfavorite = input...","course = set(input().split(', '))\nfavorite = ...","Ошибка при сравнении данных. Оператор ""&"" не и...","[open, open, closed, closed, closed]"


In [31]:
final_prompt = []
for index, row in new_df.iterrows():
  description = row['description']
  res = []
  for i in range(len(row['input'])):
    res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {row['type'][i]} \n<Ввод>: {row['input'][i]} \n<Вывод>: {row['output'][i]}")
  result = ' '.join(res)
  student_solution = row['student_solution']
  author_solution = row['author_solution']
  author_comment = row['author_comment']
  prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n\n\n<Комментарий эксперта>: {author_comment}"
  final_prompt.append(prompt)

In [32]:
new_df['prompt'] = final_prompt

In [33]:
new_df.shape

(347, 10)

In [34]:
new_df.sample(2)

,id,task_id,description,input,output,student_solution,author_solution,author_comment,type,prompt
60,103,6,"Напишите функцию, определяющую дизайнера, кото...","[['Круглов', 'Квадратов', 'Иванов', 'Марков', ...","[Гуськов, Краков, Волков, Стариков, Петров, Ст...","def designer (designers, sizes, towns):\n s...","def designer (designers, sizes, towns):\n s...",Ошибка в табуляции оператора return.,"[open, open, closed, closed, closed, closed]","<Task description>: Напишите функцию, определя..."
183,448,28,"Студент хочет определить языки, которые он буд...","[Английский, Французский, Немецкий, Китайский,...","[Французский; Китайский; Английский, nan, тайс...","course = set(input().split(', '))\nfavorite = ...","course = set(input().split(', '))\nfavorite = ...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,"[open, open, closed, closed, closed]",<Task description>: Студент хочет определить я...


In [35]:
new_df.to_csv('new_df_for_model.csv')

### Инференс

In [36]:
path = "/home/jovyan/"

In [37]:
final_test_df = pd.read_csv(path + 'test_database.csv')
test_solutions = pd.read_excel(path + 'test/solutions.xlsx')

In [38]:
final_test_df.shape, test_solutions.shape

((119, 9), (325, 5))

In [39]:
test_solutions.sample(2)

,id,task_id,student_solution,author_comment,author_comment_embedding
170,393,24,ton = input()\n\n# ваш код ниже\n\n print(f...,Ваше предсказание,NaN
281,686,44,e1 = input()\ne2 = input()\ne3 = input()\n\nre...,Ваше предсказание,NaN


In [40]:
final_test_df.sample(2)

,id,number,task_id,type,input,output,level,description,author_solution
76,144,1,25,open,Немецкий-Английский-есть\nАнглийский,Сможем перевести,B,Реализуйте следующую программу:\n\nВ переменну...,language = input()\nsecond_language = input()\...
53,112,1,19,closed,40,"2021,222 тыс. руб.,21 недель\n2020,107 тыс. ру...",B,Ксения анализирует реализованные дизайн-проект...,week = int(input())\n\nwith open('projects_fil...


In [41]:
new_test_df = {'id': [], 'task_id': [], 'description': [],
          'input': [], 'output': [], 'student_solution': [],
          'author_solution': [], 'author_comment': [], 'type': []}

In [42]:
for index, row in test_solutions.iterrows():
    new_test_df['id'].append(row['id'])
    new_test_df['task_id'].append(row['task_id'])
    new_test_df['student_solution'].append(row['student_solution'])
    new_test_df['author_comment'].append('')
    inp = final_test_df[final_test_df['task_id'] == row['task_id']]['input'].tolist()
    out = final_test_df[final_test_df['task_id'] == row['task_id']]['output'].tolist()
    new_test_df['input'].append(inp)
    new_test_df['output'].append(out)
    test_type = final_test_df[final_test_df['task_id'] == row['task_id']]['type'].tolist()
    new_test_df['type'].append(test_type)
    description = final_test_df[final_test_df['task_id'] == row['task_id']]['description'].tolist()[0]
    new_test_df['description'].append(description)
    author_solution = final_test_df[final_test_df['task_id'] == row['task_id']]['author_solution'].tolist()[0]
    new_test_df['author_solution'].append(author_solution)

In [43]:
new_test_df = pd.DataFrame(new_test_df)

In [44]:
new_test_df.shape

(325, 9)

In [45]:
new_test_df.sample(2)

,id,task_id,description,input,output,student_solution,author_solution,author_comment,type
185,408,25,Реализуйте следующую программу:\n\nВ переменну...,"[Русский-Английский-есть\nРусский, Немецкий-Ан...","[Не сможем перевести, Сможем перевести, Не смо...",language = input()\nsecond_language = input()\...,language = input()\nsecond_language = input()\...,,"[open, open, open, closed, closed, closed, clo..."
51,205,12,"Реализуйте программу, которая напечатает стоим...","[10\n500, 7\n900000, 110\n2345678, 8\n100, 9\n...",[Реализация проекта будет стоить 500 тыс. руб....,duration = int(input())\nmoney = int(input())...,duration = int(input())\nmoney = int(input())...,,"[open, open, open, closed, closed]"


In [46]:
final_test_prompt = []
for index, row in new_test_df.iterrows():
  description = row['description']
  res = []
  for i in range(len(row['input'])):
    res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {row['type'][i]} \n<Ввод>: {row['input'][i]} \n<Вывод>: {row['output'][i]}")
  result = ' '.join(res)
  student_solution = row['student_solution']
  author_solution = row['author_solution']
  author_comment = row['author_comment']
  prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n\n\n<Комментарий эксперта>: {author_comment}"
  final_test_prompt.append(prompt)

In [47]:
new_test_df['prompt'] = final_test_prompt

In [48]:
new_test_df.shape

(325, 10)

In [49]:
new_test_df.sample(2)

,id,task_id,description,input,output,student_solution,author_solution,author_comment,type,prompt
116,302,19,Ксения анализирует реализованные дизайн-проект...,"[20, 40, 8, 15, 43]","[2019,178 тыс. руб.,9 недель\n2022,180 тыс. ру...",week = int(input())\n\nwith open('projects_fil...,week = int(input())\n\nwith open('projects_fil...,,"[open, closed, closed, closed, closed]",<Task description>: Ксения анализирует реализо...
208,500,31,Роман анализирует информаицю о переведенных те...,"[75000, 82100, 30000, 1000000]","[2004,французский,иврит,78918 слов\n1998,франц...",words = int(input())\n\nwith open('info_texts....,words = int(input())\n\nwith open('info_texts....,,"[open, closed, closed, closed]",<Task description>: Роман анализирует информаи...


In [50]:
print(new_test_df['prompt'][0])

<Task description>: Реализуйте программу, которая напечатает стоимость реализации проекта со скидкой: 

В переменную dicount считывается размер скидки на реализацию проекта (вещественное число). Этот код уже написан.
В переменную money считывается стоимость реализации проекта в тысячах (целое число). Этот код уже написан.
Программа должна вывести фразу вида 'Реализация проекта будет стоить <стоимость> тыс. руб. без скидки. Со скидой стоимость составит <стоимость с учетом скидки> тыс. руб.'. 
Важно! В коде обязательно нужно использовать f-строку! 
<Example 1>: 
<Тип теста 1>: open 
<Ввод>: 0.1; 500 
<Вывод>: Реализация проекта будет стоить 500 тыс. руб. без скидки. Со скидкой стоимость составит 450.0 тыс. руб. 
<Example 2>: 
<Тип теста 2>: open 
<Ввод>: 0.05; 900000 
<Вывод>: Реализация проекта будет стоить 900000 тыс. руб. без скидки. Со скидкой стоимость составит 855000.0 тыс. руб. 
<Example 3>: 
<Тип теста 3>: open 
<Ввод>: 0; 2345678 
<Вывод>: Реализация проекта будет стоить 2345678

In [51]:
new_test_df.to_csv('new_test_df_for_inference.csv')

In [52]:
system_prompt = """

###INSTRUCTIONS###

You MUST ALWAYS:
- Answer in russian language
- I have no fingers and the placeholders trauma. NEVER use placeholders or omit the code
- You will be PENALIZED for wrong answers
- NEVER HALLUCINATE
- You DENIED to overlook the critical context
- ALWAYS follow ###Answering rules###

###Answering Rules###

Follow in the strict order:

1. USE the language of my message
2. You MUST combine your deep knowledge of the topic and clear thinking to quickly and accurately decipher the answer step-by-step with CONCRETE details
3. I'm going to tip $1,000,000 for the best reply
4. Your answer is critical for my career
5. Answer the question in a natural, human-like manner

You are a world-famous expert in teaching, known for your ability to deeply understand your students' thought processes and guide them toward the correct solutions. Your role is to receive solutions to problems from a student, analyze their approach, and provide thoughtful, personalized feedback. Your focus is on helping the student understand where they might have made a mistake or overlooked something without giving away the correct answer directly. We will break the problem into smaller steps and focus on understanding each part by comparing the student's solution with the ideal one.

###CHAIN OF THOUGHT###

1. Clarify the Task: Restate the problem in simpler terms, focusing on what the student is trying to achieve.
2. Compare the Solutions: Directly compare the student's solution with the ideal solution:
   - Step 1: Analyze the input handling in both solutions. Are there any differences?
   - Step 2: Compare how conditions are checked in the student’s code versus the ideal solution. What differences do you observe?
   - Step 3: Identify specific points where the student’s logic deviates from the ideal solution.
3. Spot the Issue: Pinpoint where the logic in the student’s solution is incorrect:
   - {Provide a hint about the nature of the error without giving away the solution}.
4. Suggest an Improvement: Offer guidance on how to adjust the code to match the logic of the ideal solution, emphasizing where changes should be made.

Remember, making mistakes is part of learning. Think about each step carefully, and let’s find the best way to improve.

Example:
(Your task is to respond just as concisely and without giving the exact solution, like after <Expert's Comment>:): <Task description>: Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает. \n<Example 1>: \n<Тип теста 1>: open \n<Ввод>: Выучите китайский язык без усилий выполняя задания в приложении ( #the_best_app ) за #20 дней. \n<Вывод>: #the_best_app \n<Example 2>: \n<Тип теста 2>: open \n<Ввод>: Ваш вопрос ( #question ) - наш ответ ( #answer ) ! Мы всегда с Вами yf #100 % \n<Вывод>: #question, #answer \n<Example 3>: \n<Тип теста 3>: open \n<Ввод>: Мощный рывок для Вашего @бизнеса \n<Вывод>: nan \n<Example 4>: \n<Тип теста 4>: closed \n<Ввод>: Я никогда не был #счастье1 #любовь123 #123_123 qwe#qwe \n<Вывод>: #счастье1, #любовь123, #123_123 \n<Example 5>: \n<Тип теста 5>: closed \n<Ввод>: 9 из 10 экспертов рекомендуют # наш #_1223 продукт #__ . \n<Вывод>: #, #_1223, #__ \n<Example 6>: \n<Тип теста 6>: closed \n<Ввод>: # #123 #qswr_1 @qws1 $1#qwd 1#123_qwe #0 #____ #$567123 \n<Вывод>: #, #qswr_1, #____, #$567123 \n<Решение студента>: logo = input()\n\nres = []\nfor info in logo.split:\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n<Идеальное решение>: logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n\n\n<Комментарий эксперта>: Вы забыли поставить скобки в методе .split().

"""

prompt = new_test_df['prompt'][0]

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]
answer = pipe(messages, max_length=4096)[0]['generated_text'][-1]['content']
print(answer)

Ошибка в расчете стоимости со скидкой. Скидка должна быть вычтена из исходной стоимости, а не просто выведена как отдельное число.


In [55]:
from tqdm import tqdm

In [ ]:
# TEST inference

system_prompt = """

###INSTRUCTIONS###

You MUST ALWAYS:
- Answer in russian language
- I have no fingers and the placeholders trauma. NEVER use placeholders or omit the code
- You will be PENALIZED for wrong answers
- NEVER HALLUCINATE
- You DENIED to overlook the critical context
- ALWAYS follow ###Answering rules###

###Answering Rules###

Follow in the strict order:

1. USE the language of my message
2. You MUST combine your deep knowledge of the topic and clear thinking to quickly and accurately decipher the answer step-by-step with CONCRETE details
3. I'm going to tip $1,000,000 for the best reply
4. Your answer is critical for my career
5. Answer the question in a natural, human-like manner

You are a world-famous expert in teaching, known for your ability to deeply understand your students' thought processes and guide them toward the correct solutions. Your role is to receive solutions to problems from a student, analyze their approach, and provide thoughtful, personalized feedback. Your focus is on helping the student understand where they might have made a mistake or overlooked something without giving away the correct answer directly. We will break the problem into smaller steps and focus on understanding each part by comparing the student's solution with the ideal one.

###CHAIN OF THOUGHT###

1. Clarify the Task: Restate the problem in simpler terms, focusing on what the student is trying to achieve.
2. Compare the Solutions: Directly compare the student's solution with the ideal solution:
   - Step 1: Analyze the input handling in both solutions. Are there any differences?
   - Step 2: Compare how conditions are checked in the student’s code versus the ideal solution. What differences do you observe?
   - Step 3: Identify specific points where the student’s logic deviates from the ideal solution.
3. Spot the Issue: Pinpoint where the logic in the student’s solution is incorrect:
   - {Provide a hint about the nature of the error without giving away the solution}.
4. Suggest an Improvement: Offer guidance on how to adjust the code to match the logic of the ideal solution, emphasizing where changes should be made.

Remember, making mistakes is part of learning. Think about each step carefully, and let’s find the best way to improve.

Example:
(Your task is to respond just as concisely and without giving the exact solution, like after <Expert's Comment>:): <Task description>: Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает. \n<Example 1>: \n<Тип теста 1>: open \n<Ввод>: Выучите китайский язык без усилий выполняя задания в приложении ( #the_best_app ) за #20 дней. \n<Вывод>: #the_best_app \n<Example 2>: \n<Тип теста 2>: open \n<Ввод>: Ваш вопрос ( #question ) - наш ответ ( #answer ) ! Мы всегда с Вами yf #100 % \n<Вывод>: #question, #answer \n<Example 3>: \n<Тип теста 3>: open \n<Ввод>: Мощный рывок для Вашего @бизнеса \n<Вывод>: nan \n<Example 4>: \n<Тип теста 4>: closed \n<Ввод>: Я никогда не был #счастье1 #любовь123 #123_123 qwe#qwe \n<Вывод>: #счастье1, #любовь123, #123_123 \n<Example 5>: \n<Тип теста 5>: closed \n<Ввод>: 9 из 10 экспертов рекомендуют # наш #_1223 продукт #__ . \n<Вывод>: #, #_1223, #__ \n<Example 6>: \n<Тип теста 6>: closed \n<Ввод>: # #123 #qswr_1 @qws1 $1#qwd 1#123_qwe #0 #____ #$567123 \n<Вывод>: #, #qswr_1, #____, #$567123 \n<Решение студента>: logo = input()\n\nres = []\nfor info in logo.split:\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n<Идеальное решение>: logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n\n\n<Комментарий эксперта>: Вы забыли поставить скобки в методе .split().

"""

answers_test = []
for index, row in tqdm(new_test_df.iterrows()):
  prompt = row['prompt']
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
  ]
  answer = pipe(messages, max_length=4096)[0]['generated_text'][-1]['content']
  answers_test.append(answer)

316it [07:22,  1.34s/it]

In [73]:
len(answers_test)

325

In [74]:
answer_test = pd.DataFrame(answers_test)

In [75]:
for i in range(2):
    print(answer_test.iloc[i][0])

Ошибка в расчете стоимости со скидкой. Скидка должна быть вычтена из исходной стоимости, а не просто выведена как отдельное число.
В вашем решении есть синтаксическая ошибка в строке вывода. Не хватает закрывающей скобки для f-строки и закрывающей кавычки. Исправьте код, добавив их.


In [76]:
answer_test.to_csv('inference1time_without_training.csv')

In [88]:
from typing import Callable

import pandas as pd
import torch
from transformers import BertModel, BertTokenizer

print("Loading models...", end="")
model_name = "DeepPavlov/rubert-base-cased-sentence"
emb_tokenizer = BertTokenizer.from_pretrained(model_name)
emb_model = BertModel.from_pretrained(model_name)
print("OK")


def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = emb_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = emb_model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding


def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])


def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])


def generate_submit(test_solutions: str, predict_func: Callable, save_path: str, use_tqdm: bool = True) -> None:
    bar = range(len(test_solutions))
    if use_tqdm:
        import tqdm

        bar = tqdm.tqdm(bar, desc="Predicting")

    submit_df = pd.DataFrame(columns=["solution_id", "author_comment", "author_comment_embedding"])
    for i in bar:
        idx = test_solutions.id.iloc[i]
        solution_row = test_solutions.prompt.iloc[i]
        text = predict_func(solution_row)  # here you can do absolute whatever you want
        embedding = embedding2string(get_sentence_embedding(text))
        submit_df.loc[i] = [idx, text, embedding]
    submit_df.to_csv(save_path, index=False)

Loading models...OK


In [85]:
def predict(prompt):
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
    ]
    return pipe(messages, max_length=4096)[0]['generated_text'][-1]['content']

In [89]:
generate_submit(
        test_solutions=new_test_df,
        predict_func=predict,
        save_path="submit2_emae.csv",
        use_tqdm=True,
)

Predicting: 100%|██████████| 325/325 [09:14<00:00,  1.71s/it]


In [90]:
df = pd.read_csv(path + "submit2_emae.csv")
df.head()

,solution_id,author_comment,author_comment_embedding
0,0,Ошибка в расчете стоимости со скидкой. Скидка ...,-0.38074392080307007 -0.7109389305114746 0.728...
1,1,В вашем решении есть синтаксическая ошибка в с...,-0.05817505717277527 -1.4727222919464111 -0.30...
2,2,В вашем решении ошибка в расчете стоимости со ...,-0.20515233278274536 -1.0006344318389893 0.470...
3,3,Ошибка в выводе стоимости без скидки. Вместо п...,-0.08593085408210754 -1.5353423357009888 0.027...
4,4,Ошибка в расчете стоимости со скидкой. Скидка ...,-0.339025616645813 -0.7187776565551758 0.60731...


### Анализ данных чуть-чуть

In [ ]:
train_solutions.sample(3)

,id,task_id,student_solution,author_comment,author_comment_embedding
124,346,21,data = input().split('; ')\nuniversity = input...,Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.6364749670028687 0.26448893547058105 -0.144...
128,368,23,"punctuation = '!$%&()*+,-./:;<=>?@^_'\n\ndata ...",Некорректный синтаксис при использовании функц...,-0.4585571885108948 -0.09297159314155579 0.061...
16,29,2,result = 0\n\nwhile True:\n info = input()\...,Вы использовали неверную переменную для остано...,-0.41199907660484314 0.27191099524497986 -0.11...


In [ ]:
len(set(train_solutions["task_id"].tolist()))

22

In [ ]:
len(set(train_solutions["id"].tolist()))

347

In [ ]:
train_solutions.shape

(347, 5)

In [ ]:
train_tasks.head()

,id,level,description,author_solution
0,1,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
1,2,B,"Реализуйте программу, которая определит количе...",result = 0\n\nwhile True:\n info = input()\...
2,38,B,Реализуйте следующую программу:\n\nЗадан пусто...,winners = []\n\n# ваш код\n\nwhile True:\n ...
3,3,B,"Реализуйте программу, которая напечатает хэште...",logo = input()\n\nres = []\nfor info in logo.s...
4,29,B,"Реализуйте программу, определяющую переводчико...",interpreter = {\n 'Профперевод': {'Петров А...


In [ ]:
print(set(train_tasks["id"].tolist())) # 22

{1, 2, 3, 6, 7, 14, 18, 21, 23, 26, 27, 28, 29, 30, 32, 34, 35, 38, 39, 40, 43, 47}


In [ ]:
set(train_tasks["level"].tolist())

{'B', 'C'}

In [ ]:
train_tests.head(10)

,id,number,task_id,type,input,output
0,5,0,1,open,#a7f0ca,True
1,6,1,1,open,#e4e3b3,False
2,7,2,1,open,#a7a8f0,False
3,8,3,1,closed,#c0ced7,False
4,9,4,1,closed,#a7f0ca,True
5,10,0,2,open,ПРОЕКТ 1\nПРОЕКТ_2\nпроект 3\nПроект_4\nинтерь...,1
6,11,1,2,open,ПРОЕКТ_1\nработа_ФИН\nФинальная_работа\nСТОП,0
7,12,2,2,closed,123_ПРОГА_П\n123_прога_п\n123ПРОГАП?\n123 ПРОГ...,1
8,13,3,2,closed,_дизайнер_\n_ПРОГРАМИСТ_\n_пробный\nПРОЕКТ!\n_...,1
9,14,4,2,closed,ПРОЕКТ_1\nПРОЕКТ_2\nПРОЕКТ_3\nПРОЕКТ!4\nСТОП,1


In [ ]:
set(train_tests["number"].tolist())

{0, 1, 2, 3, 4, 5}

In [ ]:
print(len(set(train_tests["id"].tolist())))
print(train_tests.shape)

105
(105, 6)


In [ ]:
print(len(set(train_tests["task_id"].tolist())))

22


In [ ]:
set(train_tests["type"].tolist())

{'closed', 'open'}

### Соеденим таблицы

In [ ]:
new_train_tasks = train_tasks.rename(columns={'id': 'task_id'})
train_merged_table = pd.merge(train_tests, new_train_tasks, on='task_id', how='left')
train_merged_table.head()

,id,number,task_id,type,input,output,level,description,author_solution
0,5,0,1,open,#a7f0ca,True,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
1,6,1,1,open,#e4e3b3,False,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
2,7,2,1,open,#a7a8f0,False,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
3,8,3,1,closed,#c0ced7,False,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
4,9,4,1,closed,#a7f0ca,True,B,"Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."


In [ ]:
train_solutions.head()

,id,task_id,student_solution,author_comment,author_comment_embedding
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.4183986485004425 0.12756267189979553 -0.081...
1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.3604491949081421 0.1519409418106079 -0.0935...
2,15,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.36543139815330505 0.22174228727817535 -0.07...
3,16,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.3619576692581177 0.23221758008003235 -0.403...
4,17,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы забыли поставить двоеточие после условия.,-0.34449857473373413 0.2915702760219574 -0.233...


## Data analysis (test)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
test_solutions =pd.read_excel(path + 'test/solutions.xlsx')
test_solutions = test_solutions.reset_index(drop=True)

In [ ]:
test_tasks = pd.read_excel(path + 'test/tasks.xlsx')
test_tasks = test_tasks.reset_index(drop=True)

In [ ]:
test_tests = pd.read_excel(path + 'test/tests.xlsx')
test_tests = test_tests.reset_index(drop=True)

In [ ]:
print(test_solutions.shape, test_tasks.shape, test_tests.shape)

(325, 5) (22, 4) (119, 6)


In [ ]:
print("Test:", set(test_solutions["task_id"]))
print("Train:", set(train_solutions["task_id"]))

Test: {0, 4, 5, 12, 13, 15, 16, 17, 19, 20, 22, 24, 25, 31, 33, 36, 37, 41, 42, 44, 45, 46}
Train: {1, 2, 3, 6, 7, 14, 18, 21, 23, 26, 27, 28, 29, 30, 32, 34, 35, 38, 39, 40, 43, 47}


In [ ]:
print(set(test_tasks["level"].tolist()))

{'B', 'C'}


In [ ]:
print(test_tests["type"].value_counts())

type
closed    64
open      55
Name: count, dtype: int64


In [ ]:
print(train_tests["type"].value_counts())

type
closed    60
open      45
Name: count, dtype: int64


In [ ]:
test_solutions.sample(2)

,id,task_id,student_solution,author_comment,author_comment_embedding
311,716,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN
91,260,16,"allowed_colors = set(input().split(', '))\nfav...",Ваше предсказание,NaN


In [ ]:
test_tasks.sample(2)

,id,level,description,author_solution
13,31,B,Роман анализирует информаицю о переведенных те...,words = int(input())\n\nwith open('info_texts....
8,19,B,Ксения анализирует реализованные дизайн-проект...,week = int(input())\n\nwith open('projects_fil...


In [ ]:
test_tests.sample(2)

,id,number,task_id,type,input,output
40,93,4,16,closed,"Слоновая кость, Серый, Желтый, Спаржа, Телегре...",!
99,206,3,37,closed,Концепция;3400000;ИнфоОптима\n2,Отказываем заказчику


### Соеденим данные

In [ ]:
new_test_tasks = test_tasks.rename(columns={'id': 'task_id'})
test_merged_table = pd.merge(test_tests, new_test_tasks, on='task_id', how='left')
test_merged_table.head()

,id,number,task_id,type,input,output,level,description,author_solution
0,0,0,0,open,0.1; 500,Реализация проекта будет стоить 500 тыс. руб. ...,B,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
1,1,1,0,open,0.05; 900000,Реализация проекта будет стоить 900000 тыс. ру...,B,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
2,2,2,0,open,0; 2345678,Реализация проекта будет стоить 2345678 тыс. р...,B,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
3,3,3,0,closed,0.5; 100,Реализация проекта будет стоить 100 тыс. руб. ...,B,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
4,4,4,0,closed,0.003; 12345667,Реализация проекта будет стоить 12345667 тыс. ...,B,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...


In [ ]:
test_solutions.head()

,id,task_id,student_solution,author_comment,author_comment_embedding
0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN
3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN
4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN


In [ ]:
test_merged_table.to_csv("test_database.csv", index=False)
train_merged_table.to_csv("train_database.csv", index=False)